In [1]:
import pandas as pd

In [2]:
payment = pd.read_csv("../../../downloads/olist_redefined/payment.tsv", sep='\t').drop_duplicates()
order_item = pd.read_csv("../../../downloads/olist_redefined/order_item.tsv", sep='\t').drop_duplicates()

In [3]:
order_item.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [4]:
order_item['total_selling_price'] = order_item['price'] + order_item['freight_value']
order_item.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,total_selling_price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,218.04


In [5]:
selling_price = order_item.groupby('order_id').agg({'total_selling_price':'sum'})
selling_price.head()

,total_selling_price
order_id,
00010242fe8c5a6d1ba2dd792cb16214,72.19
00018f77f2f0320c557190d7a144bdd3,259.83
000229ec398224ef6ca0657da4fc703e,216.87
00024acbcdf0a6daa1e931b038114c75,25.78
00042b26cf59d7ce69dfabb4e55b4fd9,218.04


In [6]:
# 'count' 함수를 직접 사용
total_payment = payment.groupby('order_id', as_index=False).agg({'payment_value': 'sum'})
total_payment.rename(columns={'payment_value':'total_payment'}, inplace=True)
total_payment.head()

,order_id,total_payment
0,00010242fe8c5a6d1ba2dd792cb16214,72.19
1,00018f77f2f0320c557190d7a144bdd3,259.83
2,000229ec398224ef6ca0657da4fc703e,216.87
3,00024acbcdf0a6daa1e931b038114c75,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04


In [7]:
comp_price = total_payment.merge(selling_price, on='order_id', how='left')

In [8]:
comp_price.shape

(99440, 3)

In [9]:
comp_price['diff'] = round(comp_price['total_selling_price'] - comp_price['total_payment'], 2)
comp_price

,order_id,total_payment,total_selling_price,diff
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,72.19,0.0
1,00018f77f2f0320c557190d7a144bdd3,259.83,259.83,0.0
2,000229ec398224ef6ca0657da4fc703e,216.87,216.87,0.0
3,00024acbcdf0a6daa1e931b038114c75,25.78,25.78,0.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04,218.04,0.0
...,...,...,...,...
99435,fffc94f6ce00a00581880bf54a75a037,343.40,343.40,0.0
99436,fffcd46ef2263f404302a634eb57f7eb,386.53,386.53,0.0
99437,fffce4705a9662cd70adb13d4a31832d,116.85,116.85,0.0
99438,fffe18544ffabc95dfada21779c9644f,64.71,64.71,0.0


In [10]:
comp_price['diff'].value_counts()

diff
 0.00     98089
 0.01       170
-0.01       103
-0.02        20
 0.02        16
          ...  
-12.11        1
-3.66         1
-14.99        1
-9.26         1
 21.80        1
Name: count, Length: 255, dtype: int64

In [11]:
comp_price[comp_price['diff'] > 0.0]

,order_id,total_payment,total_selling_price,diff
1807,04993613aee4046caf92ea17b316dcfb,524.28,524.32,0.04
1992,051fcda88d997d3ff86012da2a556342,51.70,56.60,4.90
2589,0693ed674763a4fc34e2c763c7fd25c2,642.62,642.64,0.02
3395,08af87fbe33eeebc7840433d94718046,296.72,296.73,0.01
3511,08ed4f84b1a078202e8da80c8d96dc77,72.80,72.81,0.01
...,...,...,...,...
98374,fd33085945f15975375cd8ec85440511,212.82,234.62,21.80
98426,fd571ab12e8d263eaf309774eedfcfeb,948.69,948.70,0.01
98764,fe35e99a6a5d4d036bf8167d364e0792,262.59,262.60,0.01
98993,fecb65750b4fe05b8257f650b2e114a2,216.75,216.76,0.01


In [12]:
price_count = order_item.groupby('product_id').agg({'price': 'nunique'})
price_count[price_count['price'] > 1]

,price
product_id,
00210e41887c2a8ef9f791ebc780cc36,2
002159fe700ed3521f46cfcf6e941c76,2
00250175f79f584c14ab5cecd80553cd,2
005030ef108f58b46b78116f754d8d38,2
007c63ae4b346920756b5adcad8095de,3
...,...
ffe0fc4e02c3559643ac063fa5cf9d07,2
ffe8083298f95571b4a66bfbc1c05524,2
fff0a542c3c62682f23305214eaeaa24,2


# 동일 제품인데 가격이 다른게 있다.
- 가격을 변경했거나
- 배송사를 변경했거나

In [13]:
diff_price = order_item.groupby(['product_id']).agg({'price': 'nunique', 'freight_value': 'nunique'})
diff_price = diff_price[(diff_price['price'] > 1) | (diff_price['freight_value'] > 1)]
diff_price

,price,freight_value
product_id,,
00126f27c813603687e6ce486d909d01,1,2
001795ec6f1b187d37335e1c4704762e,1,5
001b72dfd63e9833e8c02742adf472e3,1,9
00210e41887c2a8ef9f791ebc780cc36,2,2
002159fe700ed3521f46cfcf6e941c76,2,7
...,...,...
ffedbd68fa6f44e788ff6c2db8094715,1,2
ffef256879dbadcab7e77950f4f4a195,1,4
fff0a542c3c62682f23305214eaeaa24,2,3


In [21]:
diff_price.describe()

,price,freight_value
count,12614.000000,12614.000000
mean,1.749723,4.171318
std,1.319098,4.552754
min,1.000000,1.000000
25%,1.000000,2.000000
50%,1.000000,3.000000
75%,2.000000,4.000000
max,31.000000,86.000000


In [32]:
tmp_product_id = diff_price[diff_price['price'] == 31].index.values[0]
tmp_product_id

'437c05a395e9e47f9762e677a7068ce7'

In [37]:
# 가장 많이 가격이 변됭된 제품은 판매자가 3개다
order_item[order_item['product_id'] == tmp_product_id].nunique()

order_id               140
order_item_id            3
product_id               1
seller_id                3
shipping_limit_date    140
price                   31
freight_value           39
total_selling_price     67
dtype: int64